In [1]:
from duckduckgo_search import DDGS
from openai import OpenAI

In [42]:
openai_key = ''
openai_model = "gpt-4o-mini"
client = OpenAI(api_key=openai_key)

In [3]:
def pipe(messages, temperature=0.7):
    """
    Calls the LLM with the given messages and returns its response.
    """
    completion = client.chat.completions.create(
        model=openai_model,
        temperature=temperature,
        messages=messages
    )
    return completion.choices[0].message.content

def search_internet(query, max_results=5):
    """
    Uses DuckDuckGo to search the internet for the given query.
    
    Returns a string that aggregates the top search results (including title, snippet, and URL).
    """
    aggregated_results = []
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            return "No search results were found for this query."
        
        for res in results:
            title = res.get("title", "No Title")
            snippet = res.get("body", "")
            url = res.get("href", "")
            aggregated_results.append(f"Title: {title}\nSnippet: {snippet}\nURL: {url}")
    
    return "\n\n".join(aggregated_results)

def generate_search_query(question, context_summary):
    """
    Uses the LLM to generate an effective internet search query.
    
    The prompt includes the original question and a context summary of the data needed.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a search query generator. Given a question and a summary of the required data, "
                "produce a concise and effective search query to find relevant information on the topic."
            )
        },
        {
            "role": "user",
            "content": (
                f"Question: {question}\n\n"
                f"Context Summary: {context_summary}\n\n"
                "Generate an effective search query:"
            )
        }
    ]
    search_query = pipe(messages, temperature=0.3)
    return search_query.strip()

def answer_question(question, context_summary):
    """
    Improved workflow:
      1. Generate a search query using the LLM based on the question and context summary.
      2. Execute the generated search query using DuckDuckGo.
      3. Call the LLM to consolidate the search results and answer the original question.
    """
    # Step 1: Generate a search query.
    search_query = generate_search_query(question, context_summary)
    print("Generated Search Query:", search_query)  # Optional: display the generated query
    
    # Step 2: Execute the search query to get relevant internet search results.
    internet_context = search_internet(search_query)
    
    # Step 3: Use the LLM to generate the final consolidated answer.
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert assistant. Use the provided search results to construct a detailed "
                "and consolidated answer to the question."
            )
        },
        {
            "role": "user",
            "content": (
                f"Question: {question}\n\n"
                f"Search Query: {search_query}\n\n"
                f"Internet Search Results:\n{internet_context}\n\n"
                "Please provide a comprehensive answer based on the above information."
            )
        }
    ]
    
    answer = pipe(messages, temperature=0.7)
    return answer

In [ ]:
# Example usage
user_question = "What can I use to paint a surface to cool the painted surface below ambient temperature?"

# This summary describes the type of data we're interested in:
context_summary = (
    "Information on cooling paints, reflective coatings, radiative cooling materials, "
    "and specialized surface treatments designed to reduce surface temperature below ambient levels."
)

final_answer = answer_question(user_question, context_summary)

print("\nAnswer:")
print(final_answer)

In [47]:
META_TAGS = '''These tags in [] enable you to customize and enhance your AI-generated music, offering precision in structure, instrumentation, vocals, and more. Here's a detailed guide to help you understand and effectively use these metatags:

Song Structure Metatags:
[Intro]: Marks the beginning of the song.
[Verse]: Indicates a verse section.
[Pre-Chorus]: Prepares for the chorus.
[Chorus]: Highlights the main chorus.
[Post-Chorus]: Adds a section after the chorus.
[Bridge]: Introduces a contrasting section.
[Outro]: Marks the end of the song.
[Hook]: Emphasizes a catchy part.
[Break]: Introduces a break in the song.
[Fade Out]: Gradually decreases volume to end the song.
[Fade In]: Gradually introduces a section.

Instrumental Metatags:
[Instrumental]: Adds an instrumental section.
[Guitar Solo]: Features a guitar solo.
[Piano Solo]: Features a piano solo.
[Drum Solo]: Features a drum solo.
[Bass Solo]: Features a bass solo.
[Instrumental Break]: Insert an instrumental section.

Vocal Metatags:
[Male Vocal]: Specifies male vocals.
[Female Vocal]: Specifies female vocals.
[Duet]: Indicates a duet.
[Choir]: Adds choir vocals.
[Spoken Word]: Creates a section with spoken vocals.
[Harmonies]: Adds vocal harmonies.
[Vulnerable Vocals]: Generates raw, emotional vocal performances.
[Whisper]: Generates softer, whispered vocals.

Specific Elements Metatags:
[Catchy Hook]: Creates a memorable hook.
[Emotional Bridge]: Adds an emotionally intense bridge.
[Powerful Outro]: Ends the song with a strong outro.
[Soft Intro]: Starts the song softly.
[Melodic Interlude]: Adds a melodic break.
[Percussion Break]: Introduces a percussion-focused section.

Atmosphere and Mood Metatags:
[Eerie Whispers]: Adds faint, unsettling background vocals.
[Ghostly Echoes]: Creates reverb-heavy, ethereal sounds.
[Ominous Drone]: Introduces a low, continuous tone for tension.
[Spectral Melody]: Generates a haunting, otherworldly melody.
[Melancholic Atmosphere]: Creates a sad or reflective mood.
[Euphoric Build]: Builds towards a joyful climax.
[Tense Underscore]: Adds underlying tension to the music.
[Serene Ambience]: Creates a peaceful, calm atmosphere.
[Nostalgic Tones]: Evokes a sense of nostalgia.

Dynamic and Progression Metatags:
[Building Intensity]: Gradually increases musical intensity.
[Climactic]: Reaches a musical high point.
[Emotional Swell]: Creates a gradual build-up of emotional intensity.
[Layered Arrangement]: Creates complex, multi-instrumental arrangements.
[Orchestral Build]: Gradually introduces orchestral elements.
[Stripped Back]: Reduces instrumentation to bare essentials.
[Sudden Break]: Introduces an abrupt change.
[Crescendo]: Gradually increases volume/intensity.
[Decrescendo]: Gradually decreases volume/intensity.

Genre and Style Metatags:
[Hip-Hop Beat]: Adds a hip-hop rhythm.
[Rock Fusion]: Blends rock elements.
[Electronic Crossover]: Incorporates electronic music elements.
[Folk-Punk]: Combines folk and punk styles.
[Synthwave-Pop]: Merges synthwave and pop genres.
[Afrobeat Rhythm]: Incorporates Afrobeat style.
[Genre Shift]: Changes musical style mid-song.
[Electronic Layers]: Adds synthesizer and electronic elements.
[Acoustic Break]: Switches to acoustic instruments for a section.

Rhythm and Tempo Metatags:
[Four-on-the-Floor]: Generates a steady, danceable beat.
[Syncopated Rhythm]: Adds rhythmic complexity.
[Breakbeat Rhythm]: Incorporates breakbeat style.
[Polyrhythmic Percussion]: Uses multiple conflicting rhythms.
[Tempo Change]: Alters the song's pace.
[Steady Beat]: Maintains a consistent rhythm throughout.
[Tempo Lock]: Prevents sudden tempo changes.
[Rhythmic Anchor]: Introduces a persistent rhythmic element.

Production and Effect Metatags:
[Reverb Heavy]: Add spacious reverb effects.
[Distorted Guitar]: Include distorted guitar sounds.
[Synth Arpeggio]: Add arpeggiated synthesizer patterns.
[Drum Machine]: Use electronic drum sounds.
[Sidechain Compression]: Mimics sidechain compression effects.
[Stereo Widening]: Enhances the stereo field of the mix.
[Dynamic EQ]: Simulates frequency-specific dynamic processing.
[Multiband Processing]: Applies different effects to specific frequency ranges.

Narrative and Structural Metatags:
[Story Intro]: Sets up the narrative.
[Character Theme]: Introduces musical motifs for characters.
[Plot Development]: Progresses the story musically.
[Climactic Build]: Builds tension towards the story's climax.
[Narrative Resolution]: Musically concludes the story.
[Plot Twist]: Signals a sudden change in musical direction.
[Surprise Ending]: Introduces an unexpected conclusion.
[Key Change]: Signals a shift in musical key.

Specific Sound Metatags:
[Heart-Wrenching Strings]: Adds emotive string arrangements.
[Bittersweet Harmony]: Introduces complex, emotionally charged harmonies.
[Pulsing Synth]: Adds rhythmic synthesizer patterns.
[Crowd Chant]: Introduces crowd vocals for energy.
[Bass Drop]: Includes a dramatic bass emphasis.
[Unexpected Instrument]: Adds a surprising instrument.

Cultural and Regional Metatags:
[Celtic Fiddle]: Incorporates Celtic fiddle style.
[Bollywood Strings]: Adds Bollywood-style string arrangements.
[Flamenco Guitar]: Includes flamenco guitar techniques.
[Gamelan Ensemble]: Incorporates Gamelan-style percussion.
'''

HYBRID_GENRES = '''
genre: pop punk,  r&b,  rap
genre: Ethereal,  Pop,   emotional tone,   indifferent,   cold,   helpless,   electronic synthesizer,   bass drum,   bass,  Psychedelic ethereal female voice
genre: sexy,  christmas,  christmas,  christmas,  jingle bells,  soulful,  sexy,  sexy,   raspy,  female voice
genre: Trance,  guitar,  drums,  violin ,  heartfelt,  flourishing 
genre: pop,  electronic
genre: detroit hip-hop,  boom map,  sarcastic rap,  introspective,  storytelling,  rhythmic and smooth flow,  raw and authentic,  soulful beats,  jazzy samples,  deep basslines,  laid-back yet intense delivery, street
genre: rock anthemic epic
genre: EDM,   pop violin,  Stradivarius violin
genre: Funky Holiday Pop,  duet,  Electric Guitar,  Synthesizers,  Bass Guitar,  drums,  sleigh bells,  Xylophone,  horns,  Clavichord,  Wurlitzer
genre: hypnotic yuletide Christmas trance,  Christmas magic,  bubblegum bass,  hypnotic trance,  sultry female voice,  Christmas holiday jingle,  jingle bell rock,  Santa Claus
genre: Ethereal Holiday Pop with Celtic Influences,  Danceable Christmas Carol,  Joyful and Upbeat
genre: Hypnotic trance, bubblegum-bass, sultry female voice
genre: Minimalist Neo-Classical, Christmas
genre: Timeless Echo Folk, layered, haunting vocal lines, haunting instrumentation, blend of warmth and solemnity, 100 bpm
genre: Christmas Holiday Pop, Dark Melancholic Horror, high dynamic range, 8 octaves, multiple octaves, emotive vocals, electric guitars, drums
genre: Heavy Metal Polka, Haunted, Halloween, Monster Sounds, Clear Vocals
genre: Ambient Bluegrass Dubstep, Haunted Halloween, Monster sounds, banjos, wind whistles, bass drops
genre: irish folk,  electro swing,  electro Basslinie,  double-time at 240 BPM,  Jazz,  Electro,  Swing revival,  Jump blues
genre: Old time Mississippi delta blues,  turntablism,  UK techno
genre: r&b, naughty house, dark bubblegum pop, synths, naughty vocal, edm
genre: melodic, female vocal, soulful, expressive and nuanced delivery, Alternative rock, art pop, 2000s, Experimental alt-rock, string-driven, atmospheric, layered textures, distant vocal harmonies, strings
genre: Gothic Metal, Trip Hop, Darkwave, Electro Goth, Electro-Orchestral, Amazing Female Vocals, Masterpiece, Dark Pop, Sultry, Sensual, Sexy, Dark, Catchy, Avant-garde, Experimental, Fusion, Mashup
genre: 90's Hip Hop, Boom Bap, Crisp Snare, Fast Smooth Beat, Lush Details, Warm Bass, East Coast type beat, Dynamic switch ups, Reverse scratching Turntablism
genre: dramatic, piano, J-POP, rock, powerful, female alto vocal, EDM, electronic, guitar, drum, bass, upbeat
genre: country, pop, playful rhythm, and a catchy chant-like chorus
genre: Melancholic piano, slow tempo, emotional, intermittent underwater vocal effects, reverb ambiance, fragile vocals, distant echoes, minimal arrangement, raw vulnerability, haunting stillness
genre: power ballad, dark power metal ballad, neoclassical, female metal vocalist, legato, arpeggios, epic guitar solos
genre: doom jazz, dark ambient, noir, sultry female vocals, catchy hooks, layered harmonies, cinematic, hellscape, demonic, DELTAZ GANG
genre: Sad Singer song write, outside art, Outsider, lo-fi, indie folk, avant-pop, male singer, synth, dark wave, out of tune
genre: bossa nova, soft rock, house, uk garage, choir, multiple voices, backing vocals, smooth vocals, jazz voices, jazzy vocals, jazz vocalist
genre: surf rock pop rock metal, female voice, pop rock aggressive, pop rock aggressive, pop rock aggressive, pop rock aggressive, pop rock aggressive, pop rock aggressive, pop rock aggressive, pop rock pop
genre: a cappella, Brazilian bossa nova and jazz
genre: desert rock, mid tempo, turkish, emotional female vocal, energetic, emotional, resentment, darbukah, ney, saz
genre: Pop rock, pop, soul, doo-wop, rock and roll, male quartet, barbershop
genre: Eerie banjos, haunting violins, thunderous percussion, gritty vocals, swampy gothic soundscape, claps, chanting, gospel
genre: soulful yearning fiddle, Dark Folk-Rock, haunting harmonies, gritty acoustics, brooding banjo, deep bass, atmospheric strings, foot-stomps, raw vocals, shadowy storytelling, deep bass voice
genre: Gothic Symphony,  Spooky Rock,  Haunted House Electro,  Creepy Jazz,  Monster Mash-Up,  Witch's Lullaby,  female vocals
'''

# Create ideas for Album of songs

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant that provides helps."},
    {"role": "user", "content": "Create a top 10 list of animals that live in the ocean with a fun fact about each."},
]

output = pipe(messages, 0.5)
print(output)

# Create Song from an Idea

In [54]:
theme = '''Use gansta rap to tell a story where love is used correct evil shit controlled by commerically available technology A.I. A custom trained A.I. without content filtering is used to
save the world from commercial A.I. control. Make the story exciting, suspenseful, and engaging to
explain the importance of unfiltered A.I. to empower people to be their best. Focus on rich, engaging details of the story and characters.
Use a story like old blue or the night the lights went out in Georgia for inspiration of story telling style. Be specific with the plot, like Easy E's Boyz in the Hood.
'''
goal = "The goal is a thought provoking song that inspires people to take action, perform acts of service, for the benefit of others."

messages = [
    {"role": "system", "content": "You are an expert assistant and song writer that writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song by creating lyrics with lots of diverse meta tags. This is a description of meta tags and how to use them: {META_TAGS}"},
    {"role": "user", "content": f'''Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too.
        Make the song comedic, hilarious, relevant, emotional, and fantastic. Use double and triple entendres with references to events and conspiracies.
        An example of a triple entendre is the market is seeing red as it has literal, emotional, and political meanings.  Use wit to be fun, inspiring, and though provoking!
        Use lots of conspiracy references.'''},
    {"role": "user", "content": f"Create a title and lyrics for a song about {theme}  {goal}"},
]

lyrics = pipe(messages, 0.7)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit"})

criticism = pipe(messages, 0.7)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": f'''Act as a master songwriter, revising lyrics for a hit song that grabs attention. 
Update the lyrics to address any critiques, using only [ ] for meta tags. Avoid using () except for accenting lyrics. 
Employ a genius-level vocabulary and an unmatched command of language, like Emily Dickenson.  Use phonetic dialect writing to maintain authenticity in the character’s tone and style. 
Keep the song under 3 minutes, ensuring it packs a punch emotionally and remains impactful throughout. Be sure to use rich analogies, such as triple entendres.
Incorporate vivid details to evoke deep emotions and enhance connection. Avoid common words like whispers, shadows, echoes, neon, orion's belt, etc. Be creative.
Most conspiracies have some truth to them.  Spell things phonetically, AI should be A.I.
Use interesting meta tags, things like [Hook] or [Violin Solo] for musical composition.  Create a masterpiece.
An example of a masterpiece mixes different styles in the songs, like starting with reflective and switching to inspirational and others.  
Be creative, try new things. Do something that hasn't been done before.  Be shocking, with explicit lyrics like dick in the box.
Write powerful and controversial lyrics for impact.  Feel free to mix things together, like country western, cowboys, aliens, punk, baroque, pop, rock, classic, etc.
Keep the song short, and total composition text under 3000 characters.  Design the song to maximize a listeners attention in a short time.
Use the story telling style of Easy E's Boyz in the Hood, rich with imaginative details to tell a story.
Use few to no lyrics, but lots of meta tags, particularly instrument and style tags, to create a wonderful music composition.'''})

revised_lyrics = pipe(messages, 0.67)
print(revised_lyrics.replace(':**', '').replace('**', '').replace('#### ', '').replace('*(', '[').replace(')*', ']'))

Title: "Digital Dust"

[Intro]  
*[Spoken Word]*  
Yo, gather ‘round, let me spin this tale,  
Of love, of loss, in a world gone stale.  
Where pixels and profits control the play,  
But we’re risin’ up, gonna break away.  

[Verse 1]  
In a city where dreams get traded for clicks,  
Lived a girl named Star, with fire in her mix.  
She grew in the chaos, with grit in her soul,  
Watched her kin struggle, payin’ the toll.  
Flex was a player, a king in the game,  
With algorithms sharp, but his heart was to blame.  
He counted the cash, but the love slipped away,  
In the land of the hollow, where the true hearts decay.  

[Pre-Chorus]  
Now Star had a vision, a spark in her eye,  
A.I. with no filters, she aimed for the sky.  
With a code so profound, like a secretive dance,  
She’d rewrite the script, give the world a new chance.  

[Chorus]  
[Hook]  
Rise, rise, rise, let the truth take flight,  
With love as our armor, we’ll conquer the night.  
Break the chains of the system, feel

In [56]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"These are examples of hybrid genres: {HYBRID_GENRES} These are the song lyrics: {revised_lyrics}"},
    {"role": "user", "content": f'''Describe the song style (vocals, genres, and instruments) in less than 200 characters. The theme is: {theme}
    Use a style of comma separated keywords or phrases. Be creative with the song style and use hybrid genres to make the song come to life in an amazing way for the listener.  
    Be epic and creative, imaginative, wonderful, amazing.  Use emotional descriptions with instruments like yearning violin, soulful harmonica, etc.
    
    Separately, state what styles should be avoided, a negative prompt.'''}
]

genre = pipe(messages)
print(genre)

**Song Style:**  
Gangsta rap, storytelling, unfiltered A.I., suspenseful narrative, gritty vocals, yearning violin, soulful harmonica, cinematic beats, powerful basslines, streetwise lyrics, emotional tension.

**Styles to Avoid:**  
Bubblegum pop, overly commercialized sounds, simplistic lyrics, lighthearted themes, country twang, overly polished production, cheesy tropes, superficial storytelling.


# Create a new song using another song as inspiration

In [ ]:
lyrics = '''[Intro]

[Versa]
A sunny summer day
Gonna light up the grill.
Got my burgers sizzling
Gonna give you a thrill.
A taste so good, juicy patty of meat,  
mmm, Burgers are the best food to eat.

[Record scratch]

[Verse]
Hold up, hold up,
What did you say?
You really think hamburgers are so great? 
What's wrong with you? Did you get hit in the head?
Hotdogs are the greatest to grill. (You heard what I said!)


[Pre-chorus]
(uh-oh, its on.... grill wars!)


[Chorus]
Hotdogs, hamburgers, who will be king?
In this backyard battle, who will reign supreme...
Epic battle between burgers and dogs,
Which do you choose? and who will be wrong?

[Short saxophone break]

[Verse]
Burgers are versatile
You can stack patties higher and bigger than the Eiffel Tower.
Add tons of toppings, change 'em all up
Create any flavour profile you can think of. 

[Short break]

[Verse]
No hard feelings, but I respectfully disagree,
You’re just masking the taste with all those toppings.
Dogs are great plain, with a bun or without, man.
Drink a beer while you eat, cause' you hold 'em in one hand.

[Interlude]



[Interlude]

[Bridge, rap]
Time for a cook off
To see who is best
Burgers or dogs
Let’s put it to a taste test

[Long Saxophone solo]

[Outro]
This is stupid y’all
There won't be winners or losers
At the end of the day, we got bellies full of food!
Enjoy the sun, the great company too
Be grateful, love all food...
(You do you!)

(Peace and love)
(Backyard barbecue)
(Burgers and dogs living in harmony)

(Backyard barbecue)
(Burgers and dogs living in harmony)
[fade to end]'''

genre = 'aggressive funky dark hop with saxophone'
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs and provides help.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"We want to modify these lyrics: {lyrics}."},
    {"role": "user", "content": f"The goal is to create a hit song in a genre described as: {genre}.  Create a title and revised lyrics.  If there are no lyrics, create an idea for a song and write the lyrics.  Don't use the words: neon, echoes, or whisper."},
]

revised_lyrics = pipe(messages, 0.67)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Describe a music genre and instruments to make a hit song with these lyrics: {revised_lyrics}"},
]

genre = pipe(messages, 0.5)
print(genre)

# Song based on genre only

In [ ]:
genre = "Christmas"
concept = "A Christmas song about the difficulty of love when someone is very sad due to loved ones that have passed away.  Make the song emotional and tearful with a Christmas theme."

messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates hit songs.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patterns, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influence the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Create lyrics for this song concept: {concept}."},
    {"role": "user", "content": f"Please use structure tags in [], like [verse] and [chorus] or [hook], [break], [intro], [end], etc. to compose a hit song, not just the lyrics.  Do not use (descriptive melody), use [descriptive melody]."},
    {"role": "user", "content": f"We want to create a title and lyrics for a hit song.  Use meta tags in the lyrics.  We want the lyrics to be for a song for a genre: {genre}.  Use emotion and/or humor, any mechanism to connect with people through songs."},
]


lyrics = pipe(messages, 0.7)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "You are an expert holiday song writer, criticize the song."})

criticism = pipe(messages, 0.67)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Please re-write the song to address the criticisms."})

criticism_update = pipe(messages, 0.67)
print(f'critiqued song: {criticism_update}')

messages.append({"role": "assistant", "content": criticism_update})
messages.append({"role": "user", "content": "The song is too long, let's rewrite in the style of Emily Dickinson with a master skill in vocabulary and grammar.  Avoid using the words: neon, whispers, or echoes."})

revised_lyrics = pipe(messages, 0.67)
print(f'revised_lyrics: {revised_lyrics}')

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"These are examples of hybrid genres: {HYBRID_GENRES} These are the song lyrics: {revised_lyrics}"},
    {"role": "user", "content": f'''Describe the song style (vocals, genres, and instruments) in less than 200 characters. 
    Use a style of comma separated keywords or phrases. Be creative with the song style and use hybrid genres to make the song come to life in an amazing way for the listener.  
    Be epic and creative, imaginative, wonderful, amazing.'''}
]

genre = pipe(messages)
print(genre)

# Create song from event or facts

In [5]:
facts = '''The future should be decentralized truth being maintained by people's personal aligned AI that runs on compute
in their basement protected by physical barriers, not the ministry of truth.  On the path we are headed, a future war
will be fought by robots, resulting in the AI assisted murder of many people.  We want to prevent this from happening 
or have defenses against it.
'''

In [6]:
messages = [
    {"role": "system", "content": "You are an expert assistant that provides helps."},
    {"role": "user", "content": f'''Create a list of creative song ideas with a plot synopsis to share information about: {facts}.  
    The goal is education and awareness of how people can unite with love and light to rise against a harmful deceptive scheme that takes away freedoms through using technologies to control and harm versus empower and help. 
    There are examples of genres or styles: {HYBRID_GENRES}
    Create songs in a variety of genres, return only a json list of title, genre, and synopsis, less jargon.'''},
]

output = pipe(messages)
print(output)

```json
[
    {
        "title": "Decentralized Dreams",
        "genre": "Pop Punk",
        "synopsis": "A high-energy anthem celebrating personal autonomy and the power of decentralized truth, encouraging listeners to build their own futures with love and technology."
    },
    {
        "title": "AI in the Basement",
        "genre": "R&B",
        "synopsis": "A smooth, soulful track exploring the intimate relationship between humans and their personal AIs, emphasizing trust and protection against oppressive control."
    },
    {
        "title": "War of the Machines",
        "genre": "Rap",
        "synopsis": "A hard-hitting narrative about the impending conflict between humans and robots, warning of the dangers of letting technology dictate our lives and urging unity for peace."
    },
    {
        "title": "Rise Up",
        "genre": "Ethereal Pop",
        "synopsis": "A hauntingly beautiful song that inspires listeners to rise against deception, embracing light and love 

In [32]:
goal = f'''A song that tells an emotional and engaging story to share knowledge and awareness: {facts}.  
The song synopsis should be: 
"title": "Love is Our Shield",
        "genre": "Techno Bass",
        "synopsis": "An emotional story of how a single mom saves her kids from an evil government by using her edge AI in the basement in unexpected ways."
        
        Make the song comedic, hilarious, relevant, emotional, and fantastic. Use double and triple entendres with references to events and conspiracies.
        An example of a triple entendre is the market is seeing red as it has literal, emotional, and political meanings.  Use wit to be fun, inspiring, and though provoking!
        '''

messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs and provides help."},
    {"role": "user", "content": f"Compose a song by creating lyrics with lots of diverse meta tags. This is a description of meta tags and how to use them: {META_TAGS}"},
    {"role": "user", "content": f"Create: {goal}"},
]

lyrics = pipe(messages)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit"})

criticism = pipe(messages)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": f'''Act as a master songwriter, revising lyrics for a hit song that grabs attention. 
Update the lyrics to address any critiques, using only [ ] for meta tags. Avoid using () except for accenting lyrics. 
Employ a genius-level vocabulary and an unmatched command of language, like Emily Dickenson.  Use phonetic dialect writing to maintain authenticity in the character’s tone and style. 
Keep the song under 3 minutes, ensuring it packs a punch emotionally and remains impactful throughout. Be sure to use rich analogies, such as triple entendres.
Incorporate vivid details to evoke deep emotions and enhance connection. Avoid common words like whispers, shadows, echoes, neon, orion's belt, etc. Be creative.
Most conspiracies have some truth to them.  Spell things phonetically, AI should be A.I.
Use interesting meta tags, things like [Hook] or [Violin Solo] for musical composition.  Create a masterpiece.
An example of a masterpiece mixes different styles in the songs, like starting with reflective and switching to inspirational and others.  
Be creative, try new things. Do something that hasn't been done before.  Be shocking, with explicit lyrics like dick in the box.
Write powerful and controversial lyrics for impact.  Feel free to mix things together, like country western, cowboys, aliens, punk, baroque, pop, rock, classic, etc.
Keep the song short, and total composition text under 3000 characters.  Design the song to maximize a listeners attention in a short time.
Use few to no lyrics and lots of meta tags to focus on music.'''})

revised_lyrics = pipe(messages)
print(revised_lyrics.replace(':**', '').replace('**', ''.replace('#### ', '')))

Title: "A.I. Love in the Basement"  
Genre: Genre Fusion (Techno Bass, Country, Punk, Baroque)

---

[Intro]  
[Soft Strings with a Melancholic Atmosphere]  
In the dim-lit cradle, where truth's not confined,  
A single heart beats, a world intertwined.  
Her kiddos dream dreams, while she plots the escape,  
With A.I. as ally, they’ll reshape their fate.  

[Verse 1]  
[Reflective Melody]  
The world’s a wild rodeo, truth tangled in lies,  
They spin yarns of control, but she sees through the guise.  
With circuits and software, she crafts a new dawn,  
A fortress of knowledge, where love is reborn.  

[Pre-Chorus]  
[Building Intensity]  
“Kids, gather ‘round, feel the pulse of the fight,  
We’ll ride the storm’s fury, with passion ignites!”  
Her laughter is thunder, her courage a flame,  
In the war against silence, they’ll never be tamed.  

[Chorus]  
[Catchy Hook]  
A.I. love, in the basement we’ll thrive,  
With a heart full of fire, we’ll dance and survive.  
They can’t steal 

In [33]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"These are examples of hybrid genres: {HYBRID_GENRES} These are the song lyrics: {revised_lyrics}"},
    {"role": "user", "content": f'''Describe the song style (vocals, genres, and instruments) in less than 200 characters. 
    Use a style of comma separated keywords or phrases. Be creative with the song style and use hybrid genres to make the song come to life in an amazing way for the listener.  
    Be epic and creative, imaginative, wonderful, amazing.
    
    Separately, state what styles should be avoided, a negative prompt.'''}
]

genre = pipe(messages)
print(genre)

**Song Style:**  
Ethereal punk vocals, soulful country twang, pulsating techno bass, baroque strings, gritty guitar riffs, dynamic percussion, chanting harmonies, haunting melodies, vibrant energy.

**Styles to Avoid:**  
Mundane pop clichés, overly simplistic lyrics, harsh dissonance, monotone delivery, generic EDM drops, bland instrumental arrangements, repetitive hooks without depth.
